## ***Machine Learning Products (MLOps)***

#### En este cuaderno jupiter se encuentra todo el proceso del Machine Learning, desde la conexión a la base de datos hasta el sistema de recomendación


#### 1. **Importamos librerias necesarias**

In [1]:
# Importamos librerias 
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import os 
import env

#### 2. **Conectamos con la base de datos**

In [2]:
# Si existen cambios en la credenciales, refrescamos el env
import importlib
import env

importlib.reload(env)

<module 'env' from 'd:\\Personal\\Documentos\\Courses\\Henry\\Henry Students\\Labs\\Cohorte 25 ft\\PG_FT_25\\Proyecto_final_Henry\\Machine_Learning\\env.py'>

In [3]:
# credenciales del AWS

HOST_DB = env.DB_HOST
USER_DB = env.DB_USER
PASS_DB = env.DB_PASS
NAME_DB = env.DB_NAME

In [4]:
# Conectamos con la base de datos 

def BD_connection(host, user_db, user_pass, name_db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user_db,
            passwd=user_pass,
            database=name_db,
            port=3306,
            connection_timeout=300
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [5]:
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)

MYSQL DATABASE connection succesful


----------------------------------------------------

In [6]:
# verifiquemos la conexion realizando una consulta simple
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    db = cursor.fetchone()
    print(f"Conectado a la base de datos: {db[0]}")

Conectado a la base de datos: UrbanTransit


In [7]:
# Funcion para obtener las tablas disponibles
def show_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    for table in tables:
        print(table)

In [8]:
# Corremos la query
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)
show_tables(connection)

MYSQL DATABASE connection succesful
('carga_registros',)
('enriched_taxi_data',)
('taxi_fhv_data',)
('taxi_zones',)
('temperaturas',)
('trafico',)


In [9]:
# Funcion para describir la estructura de una tabla
def describe_table(connection, table_name):
    cursor = connection.cursor()
    query = f"DESCRIBE {table_name}"
    cursor.execute(query)
    columns = cursor.fetchall()
    for column in columns:
        print(column)

In [10]:
describe_table(connection, 'taxi_fhv_data')

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('Pickup_datetime', 'datetime', 'NO', 'MUL', None, '')
('DropOff_datetime', 'datetime', 'NO', 'MUL', None, '')
('PULocationID', 'int', 'YES', 'MUL', '0', '')
('DOLocationID', 'int', 'YES', 'MUL', '0', '')
('trip_miles', 'float', 'YES', '', '0', '')
('driver_pay', 'float', 'YES', '', '0', '')
('VendorID', 'varchar(50)', 'YES', '', 'Unknown', '')
('source', 'char(1)', 'YES', '', 'U', '')
('trip_time', 'int', 'YES', '', '0', '')


In [11]:
# Vamos a utilizar esta funcion para correr queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query hecha con éxito")
    except Error as err:
        print("Error", err)

In [12]:
def execute_query(connection, query, fetch_data=False):
    """
    Ejecuta una consulta SQL en la base de datos conectada.

    Args:
    - connection: Conexión a la base de datos.
    - query: Consulta SQL a ejecutar.
    - fetch_data: Booleano para indicar si se deben recuperar los datos de la consulta.

    Returns:
    - Resultados de la consulta si fetch_data es True. None en caso contrario.
    """
    cursor = connection.cursor(buffered=True)  # Crear un cursor con buffering
    try:
        cursor.execute(query)  # Ejecutar la consulta

        if fetch_data:
            resultados = cursor.fetchall()  # Recuperar todos los resultados
            return resultados
        else:
            connection.commit()  # Confirmar la transacción
            print("Query hecha con éxito")

    except Error as err:
        print("Error:", err)  # Imprimir cualquier error ocurrido
    finally:
        cursor.close()  # Cerrar el cursor
        if connection.unread_result:
            connection.next_result()  # Limpiar cualquier resultado no leído
        connection.commit()  # Asegurar que cualquier resultado pendiente sea limpiado

In [ ]:
# Verificar la conexión
if connection:
    print("Conexión exitosa a la base de datos")

    # Contar registros en `users_review`
    query = "SELECT COUNT(*) FROM taxi_fhv_data;"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Número de registros en 'taxi_fhv_data': {resultado[0][0]}")

Conexión exitosa a la base de datos


In [13]:
# Crear el engine de SQLAlchemy usando la conexion MySQL
def create_sqlalchemy_engine(host_name, user_bd, password_bd, name_bd):
    url = f"mysql+mysqlconnector://{user_bd}:{password_bd}@{host_name}/{name_bd}"
    return create_engine(url)

In [14]:
# Crear el engine
engine = create_sqlalchemy_engine(HOST_DB, USER_DB, PASS_DB, NAME_DB)

In [ ]:
# Consultar SQL para obtener la tabla "taxis_fhv_data"
query_taxis = "SELECT * FROM taxi_fhv_data ORDER BY id LIMIT 1000000; "


In [18]:
# Cargamos los datos en un Dataframe
users_review = pd.read_sql(query_taxis, engine)


In [19]:
# Punto de reinicio
df_taxis = pd.DataFrame(users_review)

In [ ]:
#df_taxis.to_csv("df_taxis.csv", index=False)

In [23]:
df_taxis.head(10)

,id,Pickup_datetime,DropOff_datetime,PULocationID,DOLocationID,trip_miles,driver_pay,VendorID,source,trip_time
0,1,2024-01-01 00:46:55,2024-01-01 00:58:25,236,239,1.98,21.66,2,G,690
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,65,170,6.54,42.66,2,G,1252
2,3,2024-01-01 00:30:21,2024-01-01 00:49:23,74,262,3.08,28.05,2,G,1142
3,4,2024-01-01 00:30:20,2024-01-01 00:42:12,74,116,2.40,16.70,1,G,712
4,5,2024-01-01 00:32:38,2024-01-01 00:43:37,74,243,5.14,31.38,2,G,659
5,6,2024-01-01 00:43:41,2024-01-01 01:00:23,33,209,2.00,24.25,1,G,1002
6,7,2024-01-01 00:31:56,2024-01-01 00:48:09,74,238,3.20,28.35,1,G,973
7,8,2024-01-01 00:46:12,2024-01-01 00:57:39,166,239,2.01,24.37,2,G,687
8,9,2024-01-01 00:38:07,2024-01-01 00:39:23,226,226,0.31,6.20,2,G,76
9,10,2024-01-01 00:44:24,2024-01-01 00:57:47,7,129,2.32,20.88,2,G,803


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from dotenv import load_dotenv
import os

# 🌟 Cargar variables de entorno
load_dotenv()

# 🔐 Configuración de la conexión a la base de datos
DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'port': int(os.getenv("DB_PORT", 3306)),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'database': os.getenv("DB_NAME")
}

# Crear el motor de SQLAlchemy
engine = create_engine(
    f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# Leer los datos desde la tabla enriched_taxi_data
query = "SELECT * FROM enriched_taxi_data"
df = pd.read_sql(query, engine)

# Seleccionar las características y la variable objetivo
X = df[['PULocationID', 'pickup_borough', 'pickup_day', 'pickup_hour']]
y = df['trip_count']

# Convertir variable categórica 'pickup_borough' en variables dummy
X = pd.get_dummies(X, columns=['pickup_borough'], drop_first=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

# Función para predecir la demanda dada una zona, día y hora
def predict_demand(pulocationid, pickup_day, pickup_hour, pickup_borough):
    # Crear un DataFrame con las mismas columnas que el DataFrame original utilizado para entrenar el modelo
    input_data = pd.DataFrame({
        'PULocationID': [pulocationid],
        'pickup_day': [pickup_day],
        'pickup_hour': [pickup_hour]
    })

    # Añadir columnas dummy para 'pickup_borough'
    borough_columns = [col for col in X.columns if col.startswith('pickup_borough_')]
    for col in borough_columns:
        input_data[col] = 0
    
    # Establecer el valor adecuado de la columna dummy correspondiente al 'pickup_borough'
    if f'pickup_borough_{pickup_borough}' in input_data.columns:
        input_data[f'pickup_borough_{pickup_borough}'] = 1
    
    prediction = model.predict(input_data)
    return prediction[0]

# Ejemplo de uso de la función predict_demand
predicted_demand = predict_demand(11, 11, 13, 'Manhattan')
print(f"Predicted Demand: {predicted_demand:.2f}")


## Modelo con zonas Adjacentes

Ten en cuenta que hay que dar la ruta al archivo que contiene las zonas de taxi adjacentes

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from dotenv import load_dotenv
import os
import ast

# 🌟 Cargar variables de entorno
load_dotenv()

# 🔐 Configuración de la conexión a la base de datos
DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'port': int(os.getenv("DB_PORT", 3306)),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'database': os.getenv("DB_NAME")
}

# Crear el motor de SQLAlchemy
engine = create_engine(
    f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# Leer los datos desde la tabla enriched_taxi_data2
query = "SELECT * FROM enriched_taxi_data2"
df = pd.read_sql(query, engine)

# Leer el archivo de zonas adyacentes
adjacent_zones_df = pd.read_csv('adjacent_zones.csv')
adjacent_zones_df['adjacent_zones'] = adjacent_zones_df['adjacent_zones'].apply(ast.literal_eval)

# Función para obtener las zonas adyacentes
def get_adjacent_zones(location_id):
    row = adjacent_zones_df.loc[adjacent_zones_df['LocationID'] == location_id]
    if not row.empty:
        return row.iloc[0]['adjacent_zones']
    else:
        return []

# Seleccionar las características y la variable objetivo
X = df[['PULocationID', 'pickup_weekday', 'pickup_hour']]
y = df['trip_count']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

# Función para predecir la demanda en una zona incluyendo las zonas adyacentes
def predict_demand_with_adjacent(pulocationid, pickup_weekday, pickup_hour):
    # Obtener las zonas adyacentes
    adjacent_zones = get_adjacent_zones(pulocationid)
    all_zones = adjacent_zones + [pulocationid]
    
    # Preparar los datos de entrada para todas las zonas
    input_data = pd.DataFrame({
        'PULocationID': all_zones,
        'pickup_weekday': [pickup_weekday] * len(all_zones),
        'pickup_hour': [pickup_hour] * len(all_zones)
    })
    
    # Realizar predicciones para todas las zonas y devolver los resultados individuales
    predictions = model.predict(input_data)
    result = {zone: prediction for zone, prediction in zip(all_zones, predictions)}
    
    return result

# Ejemplo de uso de la función predict_demand_with_adjacent
predicted_demands = predict_demand_with_adjacent(1, 6, 12)  # 260 es el PULocationID, 3 es miércoles, 13 es la hora

# Encontrar la zona con mayor demanda
max_zone = max(predicted_demands, key=predicted_demands.get)
print(f"Zone with highest demand: {max_zone}, Demand: {predicted_demands[max_zone]:.2f}")
